In [1]:
import time
import pickle
import numpy as np
from src.environment import StockTradingEnvironment
from src.utils import save_pickle, load_pickle, plot_grid
from src.learner import q_learning_learning_loop
import yfinance as yf
import random
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.optimizers import Adam
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
import logging
import multiprocessing as mp
import cProfile
import pstats
import atexit

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    device = '/GPU:0'
    logging.info("Using GPU for training")
else:
    device = '/CPU:0'
    logging.info("Using CPU for training")

2024-06-24 14:33:49,569 - INFO - Using GPU for training


In [ ]:
def fetch_stock_data(symbol, start_date, end_date, output_file):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['Close'] = stock_data['Adj Close']
    
    stock_data = stock_data.drop(columns=['Adj Close'])

    # Convert column headers to lowercase
    stock_data.columns = stock_data.columns.str.lower()

    stock_data.to_csv(output_file)
        
    return stock_data

stock_data = fetch_stock_data('AAPL', '2019-06-20', '2024-06-21', 'AAPL_data.csv')

In [2]:
def huber_loss(y_true, y_pred, clip_delta=1.0):
    """Huber loss - Custom Loss Function for Q Learning

    Links: 	https://en.wikipedia.org/wiki/Huber_loss
            https://jaromiru.com/2017/05/27/on-using-huber-loss-in-deep-q-learning/
    """
    error = y_true - y_pred
    cond = K.abs(error) <= clip_delta
    squared_loss = 0.5 * K.square(error)
    quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)
    return K.mean(tf.where(cond, squared_loss, quadratic_loss))

#### Q-Learning

In [3]:
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.write = 0

    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        if left >= len(self.tree):
            return idx
        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        idx = self.write + self.capacity - 1
        self.data[self.write] = data
        self.update(idx, p)
        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

    def update(self, idx, p):
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate(idx, change)

    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return idx, self.tree[idx], self.data[dataIdx]

class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        self.tree = SumTree(capacity)
        self.alpha = alpha
        self.capacity = capacity
        self.epsilon = 0.01

    def add(self, error, sample):
        p = (error + self.epsilon) ** self.alpha
        self.tree.add(p, sample)

    def sample(self, batch_size, beta=0.4):
        batch = []
        idxs = []
        segment = self.tree.total() / batch_size
        priorities = []
        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            s = random.uniform(a, b)
            idx, p, data = self.tree.get(s)
            batch.append(data)
            idxs.append(idx)
            priorities.append(p)
        sampling_probabilities = priorities / self.tree.total()
        is_weights = np.power(self.tree.capacity * sampling_probabilities, -beta)
        is_weights /= is_weights.max()
        return batch, idxs, is_weights

    def update(self, idx, error):
        p = (error + self.epsilon) ** self.alpha
        self.tree.update(idx, p)

In [4]:
class DoubleQLearningAgent:
    def __init__(self, env, learning_rate, discount_factor, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.q_table1 = {}
        self.q_table2 = {}

    def step(self, state):
        state_index = self._get_state_index(state)
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.env.action_space.n)
        else:
            q_values = self.q_table1.get(state_index, np.zeros(self.env.action_space.n)) + self.q_table2.get(state_index, np.zeros(self.env.action_space.n))
            return np.argmax(q_values)

    def update_qvalue(self, state, action, reward, next_state):
        state_index = self._get_state_index(state)
        next_state_index = self._get_state_index(next_state)
        if np.random.rand() < 0.5:
            best_next_action = np.argmax(self.q_table1.get(next_state_index, np.zeros(self.env.action_space.n)))
            self.q_table1[state_index] = self.q_table1.get(state_index, np.zeros(self.env.action_space.n))
            self.q_table1[state_index][action] += self.learning_rate * (
                reward + self.discount_factor * self.q_table2.get(next_state_index, np.zeros(self.env.action_space.n))[best_next_action] - self.q_table1[state_index][action]
            )
        else:
            best_next_action = np.argmax(self.q_table2.get(next_state_index, np.zeros(self.env.action_space.n)))
            self.q_table2[state_index] = self.q_table2.get(state_index, np.zeros(self.env.action_space.n))
            self.q_table2[state_index][action] += self.learning_rate * (
                reward + self.discount_factor * self.q_table1.get(next_state_index, np.zeros(self.env.action_space.n))[best_next_action] - self.q_table2[state_index][action]
            )

    def _get_state_index(self, state):
        return tuple(state.flatten())

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

class DQNAgent:
    def __init__(self, env, model_name='DQN_model', gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, learning_rate=0.001):
        self.env = env
        self.model_name = model_name
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.loss = self.huber_loss
        self.custom_objects = {"huber_loss": self.huber_loss}
        self.optimizer = Adam(lr=self.learning_rate)
        self.buffer_size = 10000
        self.batch_size = 32
        self.memory = PrioritizedReplayBuffer(self.buffer_size, 0.6)
        self.beta = 0.4
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(units=128, activation="relu", input_dim=self.env.observation_space.shape[0]))
        model.add(Dense(units=256, activation="relu"))
        model.add(Dense(units=256, activation="relu"))
        model.add(Dense(units=128, activation="relu"))
        model.add(Dense(units=self.env.action_space.n))
        model.compile(loss=self.loss, optimizer=self.optimizer)
        return model

    def huber_loss(self, y_true, y_pred):
        error = y_true - y_pred
        return tf.where(tf.abs(error) < 1.0, 0.5 * tf.square(error), tf.abs(error) - 0.5)

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        state = state.flatten().reshape(1, -1)
        next_state = next_state.flatten().reshape(1, -1)
        target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0]) if not done else reward
        td_error = abs(target - np.amax(self.model.predict(state)[0]))
        self.memory.add(td_error, (state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.env.action_space.n)
        state = state.flatten().reshape(1, -1)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self):
        if len(self.memory.tree.data) < self.batch_size:
            return
        minibatch, idxs, is_weights = self.memory.sample(self.batch_size, self.beta)
        states, actions, rewards, next_states, dones = zip(*minibatch)
        states = np.array(states).reshape(self.batch_size, -1)
        next_states = np.array(next_states).reshape(self.batch_size, -1)
        targets = self.model.predict(states)
        targets_next = self.target_model.predict(next_states)

        for i in range(self.batch_size):
            target = rewards[i] + self.gamma * np.amax(targets_next[i]) if not dones[i] else rewards[i]
            td_error = abs(target - targets[i][actions[i]])
            self.memory.update(idxs[i], td_error)
            targets[i][actions[i]] = target

        self.model.fit(states, targets, epochs=1, verbose=0)
        self.update_target_model()

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [5]:
def collect_experience(env, agent, epsilon):
    obs, _ = env.reset()
    terminated, truncated = False, False
    experiences = []

    while not terminated and not truncated:
        current_action = agent.step(obs)
        next_obs, reward, terminated, truncated, _ = env.step(current_action)
        experiences.append((obs, current_action, reward, next_obs, terminated))
        obs = next_obs

    return experiences

def adaptive_learning_rate(epoch, lr):
    if epoch % 100 == 0 and epoch:
        return lr * 0.1
    return lr

def decay_epsilon(episode, epsilon, min_epsilon, decay_rate):
    return max(min_epsilon, epsilon * decay_rate)

def q_learning_learning_loop(env, agent, learning_rate: float, discount_factor: float, episodes: int,
                             min_epsilon_allowed: float, initial_epsilon_value: float,
                             buffer_size: int = 10000, batch_size: int = 32, decay_method="exponential") -> tuple:
    epsilon = initial_epsilon_value
    epsilon_decay_factor = np.power(min_epsilon_allowed / epsilon, 1 / episodes)

    reward_across_episodes = []
    epsilons_across_episodes = []

    replay_buffer = deque(maxlen=buffer_size)

    for episode in range(episodes):
        experiences = collect_experience(env, agent, epsilon)
        replay_buffer.extend(experiences)

        if len(replay_buffer) >= batch_size:
            batch = np.array(random.sample(replay_buffer, batch_size), dtype=object)
            for b_state, b_action, b_reward, b_next_state, b_done in batch:
                b_state = tuple(b_state.flatten())
                b_next_state = tuple(b_next_state.flatten())
                if b_done:
                    target = b_reward
                else:
                    target = b_reward + discount_factor * np.max(agent.q_table1.get(b_next_state, np.zeros(env.action_space.n)) + agent.q_table2.get(b_next_state, np.zeros(env.action_space.n)))

                agent.q_table1[b_state] = agent.q_table1.get(b_state, np.zeros(env.action_space.n))
                agent.q_table1[b_state][b_action] += learning_rate * (target - agent.q_table1[b_state][b_action])

        if decay_method == "exponential":
            epsilon = decay_epsilon(episode, epsilon, min_epsilon_allowed, epsilon_decay_factor)
        else:
            epsilon = max(min_epsilon_allowed, epsilon - (initial_epsilon_value - min_epsilon_allowed) / episodes)

        reward_across_episodes.append(sum([exp[2] for exp in experiences]))
        epsilons_across_episodes.append(epsilon)

        agent.decay_epsilon()  # Apply epsilon decay after each episode

        if (episode + 1) % 100 == 0:
            logging.info(f"Episode {episode + 1}/{episodes} - Reward: {reward_across_episodes[-1]} - Epsilon: {epsilon}")

    logging.info("Trained Q-Table: %s", agent.q_table1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(reward_across_episodes, label='Rewards')
    plt.xlabel('Episodes')
    plt.ylabel('Cumulative Reward')
    plt.title('Rewards over Episodes')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epsilons_across_episodes, label='Epsilon')
    plt.xlabel('Episodes')
    plt.ylabel('Epsilon')
    plt.title('Epsilon Decay over Episodes')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return agent, reward_across_episodes, epsilons_across_episodes

In [6]:
def parallel_collect_experience(env, agent, epsilon, num_workers=8):
    with mp.Pool(num_workers) as pool:
        results = pool.starmap(collect_experience, [(env, agent, epsilon) for _ in range(num_workers)])
    experiences = [exp for result in results for exp in result]
    return experiences

In [ ]:
# NOTE: You can adjust the parameter 'number_of_days_to_consider'

# env = stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)

In [7]:
import optuna

def train_agent(trial):
    # Suggest parameters
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    episodes = trial.suggest_categorical('episodes', [1000, 5000, 10000, 50000])
    days_to_consider = trial.suggest_categorical('days_to_consider', [10, 20, 30, 40])
    buffer_size = trial.suggest_categorical('buffer_size', [10000, 50000, 100000, 200000])
    
    # Initialize the environment and agent with the current parameters
    env = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=days_to_consider)
    agent = DoubleQLearningAgent(env, learning_rate=0.001, discount_factor=0.95)
    
    # Train the agent
    agent, reward_across_episodes, epsilons_across_episodes = q_learning_learning_loop(
        env,
        agent,
        learning_rate=0.001,
        discount_factor=0.95,
        episodes=episodes,
        min_epsilon_allowed=0.01,
        initial_epsilon_value=1,
        buffer_size=buffer_size,
        batch_size=batch_size,
        decay_method="exponential"
    )
    
    # Evaluate the agent
    total_reward = sum(reward_across_episodes)
    return total_reward

study = optuna.create_study(direction='maximize')
study.optimize(train_agent, n_trials=50)

print(f"Best Parameters: {study.best_params}")
print(f"Best Score: {study.best_value}")

In [8]:
def run_learned_policy(env, agent):
    obs, _ = env.reset()
    terminated, truncated = False, False
    total_reward = 0
    steps = 0

    while not terminated and not truncated:
        obs_tuple = tuple(obs.flatten())
        action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        steps += 1

    logging.info("Total Reward: %d, Steps: %d", total_reward, steps)
    return total_reward

In [9]:
if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()

    try:
        env, agent, reward_across_episodes, epsilons_across_episodes = train_agent()
    finally:
        profiler.disable()
        profiler.dump_stats('training_profile.prof')
        
        with open('profile_results.txt', 'w') as f:
            stats = pstats.Stats('training_profile.prof', stream=f)
            stats.sort_stats('cumulative').print_stats(20)
        
        atexit.register(profiler.disable) 

    total_reward = run_learned_policy(env, agent)

/Users/beckett/miniconda3/envs/tradeEnv/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


Columns after adding technical indicators: Index(['open', 'high', 'low', 'close', 'volume', 'volume_adi', 'volume_obv',
       'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt',
       'volume_vwap', 'volume_mfi', 'volume_nvi', 'volatility_bbm',
       'volatility_bbh', 'volatility_bbl', 'volatility_bbw', 'volatility_bbp',
       'volatility_bbhi', 'volatility_bbli', 'volatility_kcc',
       'volatility_kch', 'volatility_kcl', 'volatility_kcw', 'volatility_kcp',
       'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
       'volatility_dch', 'volatility_dcm', 'volatility_dcw', 'volatility_dcp',
       'volatility_atr', 'volatility_ui', 'trend_macd', 'trend_macd_signal',
       'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast',
       'trend_ema_slow', 'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
       'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index', 'trend_dpo',
       'trend_kst', 'trend_kst_sig', 'trend_kst_diff', 'trend_

2024-06-24 14:34:19,474 - INFO - Episode 100/25000 - Reward: 1.4284663867460345 - Epsilon: 0.9817479430199813


Market Return : 335.70%   |   Portfolio Return : 256.56%   |   
Market Return : 335.70%   |   Portfolio Return : 317.23%   |   
Market Return : 335.70%   |   Portfolio Return : 865.99%   |   
Market Return : 335.70%   |   Portfolio Return : 140.54%   |   
Market Return : 335.70%   |   Portfolio Return : 103.57%   |   
Market Return : 335.70%   |   Portfolio Return : 267.70%   |   
Market Return : 335.70%   |   Portfolio Return : 49.54%   |   
Market Return : 335.70%   |   Portfolio Return : 410.59%   |   
Market Return : 335.70%   |   Portfolio Return : 61.37%   |   
Market Return : 335.70%   |   Portfolio Return : 216.12%   |   
Market Return : 335.70%   |   Portfolio Return : 71.01%   |   
Market Return : 335.70%   |   Portfolio Return : 130.03%   |   
Market Return : 335.70%   |   Portfolio Return : 168.96%   |   
Market Return : 335.70%   |   Portfolio Return : 367.81%   |   
Market Return : 335.70%   |   Portfolio Return : 52.49%   |   
Market Return : 335.70%   |   Portfolio Retu

2024-06-24 14:34:25,432 - INFO - Episode 200/25000 - Reward: 1.4029079738046204 - Epsilon: 0.9638290236239637


...

Market Return : 335.70%   |   Portfolio Return : 395.26%   |   
Market Return : 335.70%   |   Portfolio Return : 316.31%   |   
Market Return : 335.70%   |   Portfolio Return : 377.28%   |   
Market Return : 335.70%   |   Portfolio Return : 541.30%   |   
Market Return : 335.70%   |   Portfolio Return : 372.16%   |   
Market Return : 335.70%   |   Portfolio Return : 468.56%   |   
Market Return : 335.70%   |   Portfolio Return : 309.82%   |   
Market Return : 335.70%   |   Portfolio Return : 349.73%   |   
Market Return : 335.70%   |   Portfolio Return : 388.05%   |   
Market Return : 335.70%   |   Portfolio Return : 386.50%   |   
Market Return : 335.70%   |   Portfolio Return : 336.83%   |   
Market Return : 335.70%   |   Portfolio Return : 386.15%   |   
Market Return : 335.70%   |   Portfolio Return : 417.76%   |   
Market Return : 335.70%   |   Portfolio Return : 344.53%   |   
Market Return : 335.70%   |   Portfolio Return : 352.33%   |   
Market Return : 335.70%   |   Portfolio 

2024-06-24 14:38:09,217 - INFO - Episode 3900/25000 - Reward: 1.5803939152282978 - Epsilon: 0.48752849010332244


Market Return : 335.70%   |   Portfolio Return : 357.83%   |   
Market Return : 335.70%   |   Portfolio Return : 427.73%   |   
Market Return : 335.70%   |   Portfolio Return : 397.64%   |   
Market Return : 335.70%   |   Portfolio Return : 343.37%   |   
Market Return : 335.70%   |   Portfolio Return : 385.69%   |   
Market Return : 335.70%   |   Portfolio Return : 446.93%   |   
Market Return : 335.70%   |   Portfolio Return : 371.50%   |   
Market Return : 335.70%   |   Portfolio Return : 335.39%   |   
Market Return : 335.70%   |   Portfolio Return : 609.10%   |   
Market Return : 335.70%   |   Portfolio Return : 416.46%   |   
Market Return : 335.70%   |   Portfolio Return : 340.54%   |   
Market Return : 335.70%   |   Portfolio Return : 340.20%   |   
Market Return : 335.70%   |   Portfolio Return : 344.74%   |   
Market Return : 335.70%   |   Portfolio Return : 370.82%   |   
Market Return : 335.70%   |   Portfolio Return : 329.36%   |   
Market Return : 335.70%   |   Portfolio 

2024-06-24 14:38:15,221 - INFO - Episode 4000/25000 - Reward: 1.175912384810462 - Epsilon: 0.4786300923225741


Market Return : 335.70%   |   Portfolio Return : 62.24%   |   
Market Return : 335.70%   |   Portfolio Return : 2930951.77%   |   
Market Return : 335.70%   |   Portfolio Return : 195.76%   |   
Market Return : 335.70%   |   Portfolio Return : 143.24%   |   
Market Return : 335.70%   |   Portfolio Return : 224.11%   |   
Market Return : 335.70%   |   Portfolio Return : 3021130.74%   |   
Market Return : 335.70%   |   Portfolio Return : 77.63%   |   
Market Return : 335.70%   |   Portfolio Return : 3526982.91%   |   
Market Return : 335.70%   |   Portfolio Return : 185.17%   |   
Market Return : 335.70%   |   Portfolio Return : 57.36%   |   
Market Return : 335.70%   |   Portfolio Return : 3586310.27%   |   
Market Return : 335.70%   |   Portfolio Return : 64.56%   |   
Market Return : 335.70%   |   Portfolio Return : 63.08%   |   
Market Return : 335.70%   |   Portfolio Return : 2370059.35%   |   
Market Return : 335.70%   |   Portfolio Return : 35.44%   |   
Market Return : 335.70%   

In [ ]:
def plot_grid(env, agent, reward_across_episodes: list, epsilons_across_episodes: list) -> None:
    env.train = False
    total_reward_learned_policy = [run_learned_policy(env, agent) for _ in range(30)]

    plt.figure(figsize=(15, 10))

    # Main plot
    plt.subplot(2, 2, 1)
    plt.plot(reward_across_episodes, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Training)')
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.plot(total_reward_learned_policy, 'ro')
    plt.xlabel('Episode')
    plt.ylabel('Reward Value')
    plt.title('Rewards Per Episode (Learned Policy Evaluation)')
    plt.grid()

    # Extra plots
    plt.subplot(2, 2, 3)
    plt.plot(reward_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward Per Episode (Training)')
    plt.title('Cumulative Reward vs Episode')
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.plot(epsilons_across_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Epsilon Values')
    plt.title('Epsilon Decay')
    plt.grid()

    plt.tight_layout()
    plt.show()

plot_grid(env, agent, reward_across_episodes, epsilons_across_episodes)

In [ ]:
stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    # Convert observation to a tuple
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()

In [ ]:
save_pickle(agent, 'aapl_q_learning_agent.pkl')

#### Forecast

In [ ]:
agent = load_pickle("aapl_q_learning_agent.pkl")

In [ ]:
stock_trading_environment = StockTradingEnvironment('./AAPL_data.csv', number_of_days_to_consider=30)
stock_trading_environment.train = False
obs, _ = stock_trading_environment.reset()
terminated, truncated = False, False
while not terminated:
    # Convert observation to a tuple
    obs_tuple = tuple(obs.flatten())
    action = np.argmax(agent.q_table1.get(obs_tuple, np.zeros(env.action_space.n)) + agent.q_table2.get(obs_tuple, np.zeros(env.action_space.n)))
    obs, reward, terminated, truncated, info = stock_trading_environment.step(action)

stock_trading_environment.render()